In [1]:
%load_ext autoreload
%autoreload 2

import scipy
import numpy as np
import tensorflow as tf
import keras
import tensorflow_probability as tfp
import matplotlib.pyplot as plt


2024-06-17 14:40:41.291808: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 14:40:41.341541: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 14:40:41.341579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 14:40:41.342753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-17 14:40:41.350523: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Distribution Settings
scale = 1
mean = 0

normal_dist = scipy.stats.norm(loc=mean, scale=scale)

# Number of independent datapoints 
T = 1000

seed=360
rng = np.random.RandomState(seed)
y_T = normal_dist.rvs(size=T, random_state=rng)

# create keras model
inputs = keras.Input(shape=0)
# a dense layer with an empty input will just be an intercept term
intercept_layer = keras.layers.Dense(1, activation = None)
# Create a normal distribution with a mean equal to the input 
distribution_layer = tfp.layers.DistributionLambda(lambda t: tfp.distributions.Normal(loc=t, scale=1))
predicted_mean = intercept_layer(inputs)
distribution = distribution_layer(predicted_mean)
model = keras.Model(inputs=inputs, outputs=distribution)

2024-06-17 14:41:03.497007: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-06-17 14:41:03.497051: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2024-06-17 14:41:03.497057: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: s1cmp008.pax.tufts.edu
2024-06-17 14:41:03.497157: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-06-17 14:41:03.497188: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-06-17 14:41:03.497194: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [4]:
x_TF = np.zeros(shape=(T,0))

In [ ]:
# number of score function trick samples
N=2
@tf.custom_gradient
def get_action(model_distribution):
    model_distribution = model(x)
    sample_y_NT = model_distribution.sample(N)
    expected_action_T = tf.reduce_mean(sample_y_NT, axis=0)

    def grad(upstream):
        with tf.GradientTape() as tape:
            sample_y_NT = model_distribution.sample(N)
            stopped_samples_NT = tf.stop_gradient(sample_y_NT)
            sample_log_probs_NT = model_distribution.log_prob(stopped_samples_NT)
            estimator = tf.reduce_mean(sample_log_probs_NT*tf.expand_dims(expected_action_T,axis=0), axis=0)
        dz_dx = tape.jacobian
        return upstream * dz_dx
    return expected_action_T, grad

In [6]:
grads = []
vals = []
for val in np.linspace(-10,10,11):
    print(val)
    intercept = model.trainable_variables[1]
    intercept.assign([val])
    for i in range(20):
        if i%10==0:
            print(i) 
        

        with tf.GradientTape() as tape, tf.GradientTape() as loss_tape:
            model_distribution = model(x_TF)
            sample_y_NT = model_distribution.sample(N)
            stopped_samples_NT = tf.stop_gradient(sample_y_NT)
            sample_log_probs_NT = model_distribution.log_prob(stopped_samples_NT)
            #estimator = tf.reduce_mean(sample_log_probs_NT*stopped_samples_NT, axis=0)
        
            sample_y_NT = model_distribution.sample(N)
            #stopped_samples_NT = tf.stop_gradient(sample_y_NT)
            expected_action_T = tf.reduce_mean(sample_y_NT, axis=0)
            estimator = tf.reduce_mean(sample_log_probs_NT*tf.expand_dims(expected_action_T,axis=0), axis=0)
            loss_T = keras.losses.mean_squared_error(y_T, expected_action_T)
            #estimator = tf.reduce_mean(sample_log_probs_NT*tf.expand_dims(loss_T,axis=0), axis=0)
        action_grad = tape.gradient(estimator, model.trainable_weights)



        other_grad = loss_tape.gradient(loss_T, expected_action_T)



        #gradients = tape.gradient(estimator, model.trainable_weights)
        grads.append(tf.reduce_mean(tf.squeeze(other_grad) * tf.squeeze(action_grad[1])))
        vals.append(val)




-10.0
0
10
-8.0
0
10
-6.0
0
10
-4.0
0
10
-2.0
0
10
0.0
0
10
2.0
0
10
4.0
0
10
6.0
0
10
8.0
0
10
10.0
0
10
